<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [8]:
import pandas as pd


train_df = pd.read_csv('/content/train.csv')
submission_df = pd.read_csv('/content/sample_submission.csv')

display(train_df.head())
display(submission_df.head())

,일시,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온
0,1960-01-01,2.2,-5.2,7.4,NaN,68.3,1.7,6.7,NaN,NaN,-1.6
1,1960-01-02,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,NaN,NaN,-1.9
2,1960-01-03,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,NaN,NaN,4.0
3,1960-01-04,10.8,1.2,9.6,0.0,79.7,4.4,2.6,NaN,NaN,7.5
4,1960-01-05,1.3,-8.2,9.5,NaN,44.0,5.1,8.2,NaN,NaN,-4.6


,일시,평균기온
0,2023-01-01,0
1,2023-01-02,0
2,2023-01-03,0
3,2023-01-04,0
4,2023-01-05,0


In [9]:
# 날짜 데이터 변환
train_df['일시'] = pd.to_datetime(train_df['일시'])
train_df = train_df.set_index('일시')

# 데이터의 시간 간격 지정
train_df.index.freq = 'D'

# 일시 컬럼이 인덱스로 할당됩니다.
train_df.head()

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온
일시,,,,,,,,,,
1960-01-01,2.2,-5.2,7.4,NaN,68.3,1.7,6.7,NaN,NaN,-1.6
1960-01-02,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,NaN,NaN,-1.9
1960-01-03,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,NaN,NaN,4.0
1960-01-04,10.8,1.2,9.6,0.0,79.7,4.4,2.6,NaN,NaN,7.5
1960-01-05,1.3,-8.2,9.5,NaN,44.0,5.1,8.2,NaN,NaN,-4.6


In [10]:
# prophet에서 데이터를 인식하도록 일시는 ds로, target값인 평균기온은 y로 지정해줍니다.
train_df = train_df.reset_index()
train_df = train_df.rename(columns={'일시': 'ds', '평균기온': 'y'})

In [16]:
from prophet import Prophet

#모델 학습
prophet = Prophet(holidays=None)
prophet.fit(train_df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0hyn288d/gc5ng2gi.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0hyn288d/_r1dp1wk.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=18032', 'data', 'file=/tmp/tmp0hyn288d/gc5ng2gi.json', 'init=/tmp/tmp0hyn288d/_r1dp1wk.json', 'output', 'file=/tmp/tmp0hyn288d/prophet_model07699_em/prophet_model-20231215060812.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
06:08:12 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
06:08:19 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [17]:
#모델 예측
future_data = prophet.make_future_dataframe(periods = 358, freq = 'd') #periods는 예측할 기간
forecast_data = prophet.predict(future_data)
forecast_data[['ds','yhat']].tail(5)

,ds,yhat
23364,2023-12-20,0.400586
23365,2023-12-21,0.293537
23366,2023-12-22,0.161249
23367,2023-12-23,0.096136
23368,2023-12-24,-0.091857


In [18]:
submission_df['평균기온'] = forecast_data.yhat[-358:].values
submission_df

,일시,평균기온
0,2023-01-01,-1.061637
1,2023-01-02,-1.163262
2,2023-01-03,-1.279313
3,2023-01-04,-1.344326
4,2023-01-05,-1.382818
...,...,...
353,2023-12-20,0.400586
354,2023-12-21,0.293537
355,2023-12-22,0.161249
356,2023-12-23,0.096136


In [15]:
#결과 저장
submission_df.to_csv("prophet_basline_submission.csv", index=False)

display(submission_df.head())

,일시,평균기온
0,2023-01-01,-1.061637
1,2023-01-02,-1.163262
2,2023-01-03,-1.279313
3,2023-01-04,-1.344326
4,2023-01-05,-1.382818
